In [ ]:
# Gerekli kütüphanelerin kurulumu
!pip install albumentations
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!git clone https://github.com/ultralytics/yolov5
!cd yolov5 && pip install -r requirements.txt

In [ ]:
# Önce Google Drive'ı bağlayalım
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!nvidia-smi


Wed Dec 25 14:15:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
print(torch.cuda.is_available())  # True dönerse CUDA destekleniyor
print(torch.cuda.device_count())  # Kullanılabilir GPU sayısını döner
print(torch.cuda.get_device_name(0))  # GPU'nuzun adını döner


True
1
Tesla T4


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import numpy as np
from torchvision import transforms
from pathlib import Path
import cv2
import time
import logging
from datetime import datetime
from typing import Dict, List, Tuple, Optional
import albumentations as A
from torch.cuda.amp import autocast, GradScaler

# Logging configuration
# Bu kısım, loglama işleminin önce time sonra name sonra levelname ve sonra mesaj şeklinde yapılacağını belirtiyor.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class Config:  # Yaml dosyasını ayı bir dosya olarak koyamayacağımdan ötürü colabda bu şekilde bir class oluşturduk.
    """Configuration class for Colab environment"""
    def __init__(self):
        # Training parameters
        self.BATCH_SIZE = 16
        self.IMG_SIZE = 640
        self.EPOCHS = 50
        self.WORKERS = 2
        self.LEARNING_RATE = 0.01

        # Model parameters
        self.ARCHITECTURE = "yolov5m"
        self.NUM_CLASSES = 7
        self.CONF_THRESHOLD = 0.25
        self.CLASS_NAMES = ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']

        # Paths - Colab specific
        self.TRAIN_IMAGES = '/content/drive/MyDrive/train/images'
        self.TRAIN_LABELS = '/content/drive/MyDrive/train/labels'
        self.VAL_IMAGES = '/content/drive/MyDrive/valid/images'
        self.VAL_LABELS = '/content/drive/MyDrive/valid/labels'
        self.OUTPUT_DIR = '/content/drive/MyDrive/model_outputs'

        # Create output directory
        os.makedirs(self.OUTPUT_DIR, exist_ok=True)


    def validate_paths(self):
        """Validate that all required paths exist"""
        paths = [self.TRAIN_IMAGES, self.TRAIN_LABELS,
                self.VAL_IMAGES, self.VAL_LABELS]

        for path in paths:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Path not found: {path}")

        logger.info("All paths validated successfully")



class YOLODataset(Dataset):
    """Custom Dataset for YOLO training with coordinate validation"""
    def __init__(self,
                 images_dir: str,
                 labels_dir: str,
                 img_size: int = 640,
                 transform: Optional[A.Compose] = None,
                 is_training: bool = True):
        self.images_dir = Path(images_dir)
        self.labels_dir = Path(labels_dir)
        self.img_size = img_size
        self.transform = transform
        self.is_training = is_training

        self.image_files = sorted([f for f in self.images_dir.glob('*')
                                 if f.suffix.lower() in ('.jpg', '.jpeg', '.png')])
        self.label_files = []

        # Etiket dosyalarını kontrol et ve eşle
        for img_file in self.image_files:
            label_file = self.labels_dir / f"{img_file.stem}.txt"
            if label_file.exists():
                # Etiket dosyasının formatını kontrol et
                try:
                    self._validate_label_file(label_file)
                    self.label_files.append(label_file)
                except ValueError as e:
                    logger.warning(f"Skipping invalid label file {label_file}: {str(e)}")
                    continue
            else:
                logger.warning(f"Missing label file for {img_file}")
                continue

        self.image_files = [img for i, img in enumerate(self.image_files)
                           if i < len(self.label_files)]

        if not self.image_files:
            raise FileNotFoundError("No valid image-label pairs found")

        logger.info(f"Found {len(self.image_files)} valid image-label pairs")

    def _validate_label_file(self, label_path: Path) -> None:
        """YOLO format etiketlerini doğrula"""
        with open(label_path, 'r') as f:
            for line_num, line in enumerate(f, 1):
                try:
                    values = [float(x) for x in line.strip().split()]
                    if len(values) != 5:
                        raise ValueError(f"Each line must have 5 values, got {len(values)}")

                    class_id, x_center, y_center, width, height = values

                    # class_id tam sayı olmalı
                    if not float(class_id).is_integer():
                        raise ValueError(f"Class ID must be integer, got {class_id}")

                    # Koordinatlar 0-1 arasında olmalı
                    for name, value in [("x_center", x_center), ("y_center", y_center),
                                      ("width", width), ("height", height)]:
                        if not 0 <= value <= 1:
                            raise ValueError(f"{name} must be between 0 and 1, got {value}")

                except ValueError as e:
                    raise ValueError(f"Invalid format in {label_path} at line {line_num}: {str(e)}")

    def __len__(self) -> int:
        return len(self.image_files)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        try:
            # Görüntüyü yükle
            img_path = str(self.image_files[idx])
            image = read_image(img_path).float() / 255.0

            if image.shape[0] == 1:
                image = image.repeat(3, 1, 1)

            if image.shape[1] != self.img_size or image.shape[2] != self.img_size:
                resize_transform = transforms.Resize((self.img_size, self.img_size))
                image = resize_transform(image)

            # Etiketleri yükle ve doğrula
            label_path = str(self.label_files[idx])
            boxes = []
            with open(label_path, 'r') as f:
                for line in f:
                    values = [float(x) for x in line.strip().split()]
                    if len(values) == 5:
                        class_id = int(values[0])  # class_id'yi integer'a çevir
                        # Koordinatları kontrol et ve düzelt
                        x_center = np.clip(values[1], 0, 1)
                        y_center = np.clip(values[2], 0, 1)
                        width = np.clip(values[3], 0, 1)
                        height = np.clip(values[4], 0, 1)
                        boxes.append([class_id, x_center, y_center, width, height])

            boxes = torch.tensor(boxes, dtype=torch.float32)

            #print(boxes)

            if self.transform and self.is_training:
                transformed = self.transform(
                    image=image.numpy().transpose(1, 2, 0),
                    bboxes=boxes[:, 1:].numpy(),  # class_id'yi çıkar
                    class_labels=boxes[:, 0].numpy().tolist()  # class_id'leri labels olarak kullan
                )
                image = torch.from_numpy(transformed['image'].transpose(2, 0, 1))
                if transformed['bboxes'].size > 0:
                    # Dönüştürülmüş bbox'ları ve class_id'leri birleştir
                    new_boxes = np.column_stack([
                        np.array(transformed['class_labels']).reshape(-1, 1),
                        np.array(transformed['bboxes'])
                    ])
                    boxes = torch.tensor(new_boxes, dtype=torch.float32)
                else:
                    boxes = torch.zeros((0, 5))

            return image, boxes

        except Exception as e:
            logger.error(f"Error loading data at index {idx} from {img_path}: {str(e)}")
            raise

class YOLOTrainer:
    """YOLO model trainer class"""
    def __init__(self):
        self.config = Config()
        self.config.validate_paths()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.scaler = GradScaler()

    def _get_transforms(self) -> A.Compose:
        """Get data augmentation transforms"""
        return A.Compose([
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.2),
            A.GaussNoise(p=0.2),
        ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

    def train(self):
        """Train the YOLO model"""
        # Setup datasets
        transform = self._get_transforms()
        train_dataset = YOLODataset(
            images_dir=self.config.TRAIN_IMAGES,
            labels_dir=self.config.TRAIN_LABELS,
            img_size=self.config.IMG_SIZE,
            transform=transform,
            is_training=True
        )

        val_dataset = YOLODataset(
            images_dir=self.config.VAL_IMAGES,
            labels_dir=self.config.VAL_LABELS,
            img_size=self.config.IMG_SIZE,
            transform=None,
            is_training=False
        )

        # Setup dataloaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.config.BATCH_SIZE,
            shuffle=True,
            num_workers=self.config.WORKERS,
            pin_memory=True,
            collate_fn=self._collate_fn
        )

        val_loader = DataLoader(
            val_dataset,
            batch_size=self.config.BATCH_SIZE,
            shuffle=False,
            num_workers=self.config.WORKERS,
            pin_memory=True,
            collate_fn=self._collate_fn
        )

        # Initialize model
        model = self._init_model()

        # Training loop
        for epoch in range(self.config.EPOCHS):
            train_metrics = self._train_epoch(model, train_loader, epoch)
            val_metrics = self._validate_epoch(model, val_loader, epoch)

            # Log metrics
            logger.info(f"Epoch {epoch}: {train_metrics} {val_metrics}")

            # Save checkpoint
            self._save_checkpoint(model, epoch, val_metrics)

    @staticmethod
    def _collate_fn(batch):
        """Custom collate function for DataLoader"""
        images = torch.stack([item[0] for item in batch])
        boxes = [item[1] for item in batch]
        return images, boxes

    def _init_model(self):
        """Initialize YOLO model"""
        model = torch.hub.load('ultralytics/yolov5',
                             self.config.ARCHITECTURE,
                             pretrained=True)
        model.nc = self.config.NUM_CLASSES
        model.to(self.device)
        return model

    def _train_epoch(self, model, dataloader, epoch: int) -> dict:
        """Train for one epoch"""
        model.train()
        metrics = {"train_loss": 0.0}

        for batch_idx, (images, targets) in enumerate(dataloader):
            images = images.to(self.device)
            #targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]
            targets = [{k[:,0]: k[:, 1:].to(self.device) for k in t.items} for t in targets]
            print(targets)
            # print(targets)
            import time
            time.sleep(999)


            with autocast():
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())

            self.scaler.scale(losses).backward()
            self.scaler.step(model.optimizer)
            self.scaler.update()
            model.optimizer.zero_grad()

            metrics["train_loss"] += losses.item()

            if batch_idx % 10 == 0:
                logger.info(f"Epoch {epoch} [{batch_idx}/{len(dataloader)}] Loss: {losses.item():.6f}")

        metrics["train_loss"] /= len(dataloader)
        return metrics

    def _validate_epoch(self, model, dataloader, epoch: int) -> dict:
        """Validate for one epoch"""
        model.eval()
        metrics = {"val_loss": 0.0}

        with torch.no_grad():
            for images, targets in dataloader:
                images = images.to(self.device)
                targets = [{k: v.to(self.device) for k, v in t.items()} for t in targets]

                outputs = model(images)
                # Calculate validation metrics

        return metrics

    def _save_checkpoint(self, model, epoch: int, metrics: dict):
        """Save model checkpoint"""
        checkpoint_path = os.path.join(self.config.OUTPUT_DIR, f"model_epoch_{epoch}.pt")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'metrics': metrics
        }, checkpoint_path)

        logger.info(f"Checkpoint saved: {checkpoint_path}")

class YOLOPredictor:
    """YOLO model predictor class"""
    def __init__(self, weights_path: str):
        self.config = Config()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self._load_model(weights_path)

    def _load_model(self, weights_path: str):
        """Load trained YOLO model"""
        try:
            model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
            model.to(self.device)
            model.eval()
            return model
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def predict_image(self, image_path: str) -> torch.Tensor:
        """Predict on a single image"""
        try:
            results = self.model(image_path)
            results.pred[0] = results.pred[0][results.pred[0][:, 4] > self.config.CONF_THRESHOLD]
            return results
        except Exception as e:
            logger.error(f"Error predicting image: {str(e)}")
            raise

    def predict_video(self, video_path: str, output_path: str, batch_size: int = 32):
        """Predict on video with batch processing"""
        try:
            cap = cv2.VideoCapture(video_path)
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

            frames = []
            frame_count = 0

            try:
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    frames.append(frame)
                    frame_count += 1

                    if frame_count % batch_size == 0:
                        batch_results = self.model(frames)

                        for results in batch_results:
                            annotated_frame = results.render()[0]
                            out.write(annotated_frame)

                        frames = []
                        torch.cuda.empty_cache()

                if frames:
                    batch_results = self.model(frames)
                    for results in batch_results:
                        annotated_frame = results.render()[0]
                        out.write(annotated_frame)

            finally:
                cap.release()
                out.release()
                cv2.destroyAllWindows()
                torch.cuda.empty_cache()

        except Exception as e:
            logger.error(f"Error processing video: {str(e)}")
            raise

def main():
    """Main function to run training and prediction"""
    try:
        # Training
        trainer = YOLOTrainer()
        trainer.train()

        # Prediction example
        weights_path = os.path.join(trainer.config.OUTPUT_DIR, "model_epoch_final.pt")
        predictor = YOLOPredictor(weights_path)

        # Example predictions
        image_path = "../input/test_image.jpg"
        if os.path.exists(image_path):
            results = predictor.predict_image(image_path)
            logger.info(f"Prediction results: {results}")

    except Exception as e:
        logger.error(f"Error in main: {str(e)}")
        raise

if __name__ == "__main__":
    main()

In [ ]:
def test_model():
    """Test fonksiyonu - eğitilmiş modeli test seti üzerinde değerlendirir ve metrikleri gösterir"""
    current_dir = os.getcwd()
    test_image_dir = '../input/test/images'
    test_label_dir = '../input/test/labels'
    train_image_dir = '../input/train/images'  # Train yolu eklendi
    val_image_dir = '../input/valid/images'    # Validation yolu eklendi

    # Test verisi için YAML config
    test_yaml = {
        'path': current_dir,
        'train': train_image_dir,     # Train yolu eklendi
        'val': val_image_dir,         # Validation yolu eklendi
        'test': test_image_dir,
        'nc': 7,
        'names': ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']
    }

    test_yaml_path = os.path.join(current_dir, 'test_data.yaml')
    with open(test_yaml_path, 'w') as f:
        yaml.dump(test_yaml, f)

    # En iyi modeli test et
    best_weights = os.path.join(current_dir, 'runs/train/yolov5_custom_model/weights/best.pt')

    # val.run kullanarak test et
    from yolov5 import val
    results = val.run(
        weights=best_weights,
        data=test_yaml_path,
        batch_size=16,
        imgsz=640,
        task='test',
        verbose=True,
        save_txt=True,
        save_conf=True,
        save_json=True,
        project=os.path.join(current_dir, 'runs', 'test'),
        name='yolov5_test_results'
    )

    # Sınıf bazlı metrikleri yazdır (eğer varsa)
    if hasattr(results, 'ap_class_index'):
        print("\nSınıf Bazlı mAP@0.5:")
        class_names = ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']
        for i, ap in enumerate(results.ap_class_index):
            print(f"{class_names[i]}: {ap:.4f}")

    return results

test_model()